# <h1>Title:</h1> Amazon Data Scraper
# <h2>Description:</h2> This Jupyter notebook illustrates the process of efficiently scraping data from Amazon's website and exporting it into a structured CSV file format.
# <h3>Author:</h3> Ahmed Hammad
# <h3>Dependencies:</h3> Beautiful Soup, Requests, Pandas
# <h3>Notes:</h3> This notebook is designed to meticulously extract valuable insights from Amazon's extensive collection of laptop listings, accessible at "https://www.amazon.com/s?k=laptop&ref=nb_sb_noss_1". It enables the comprehensive analysis and comparison of various laptop models, facilitating informed decision-making processes.


In [38]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [39]:
# Function to extract Laptop Product Title
def get_laptop_title(soup):
    try:
        # Extracting the outer tag object for the laptop title
        title = soup.find("span", attrs={"id": 'productTitle'})
        
        # Extracting the inner NavigableString object for the laptop title
        title_value = title.text

        # Converting laptop title to a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""  # In case of attribute error, return an empty string

    return title_string


In [40]:
# Function to extract Laptop Product Price
def get_laptop_price(soup):
    try:
        # Extracting the whole part of the price
        price_whole = soup.find("span", attrs={"class": 'a-price-whole'}).text
        # Extracting the fraction part of the price
        price_fraction = soup.find("span", attrs={"class": 'a-price-fraction'}).text
        # Extracting the currency symbol
        price_symbol = soup.find("span", attrs={"class": 'a-price-symbol'}).text
        # Combining all parts to form the complete price
        price = price_whole + price_fraction + price_symbol

    except AttributeError:
        price = ""  # In case of attribute error, return an empty string

    return price


In [41]:
# Function to extract Laptop Product Rating
def get_laptop_rating(soup):
    try:
        # Extracting the rating
        rating = soup.find("i", attrs={'class': 'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            # If the above class is not found, trying another class to extract rating
            rating = soup.find("span", attrs={'class': 'a-icon-alt'}).string.strip()
        except:
            rating = ""  # In case of any error, return an empty string	

    return rating

In [42]:
# Function to extract Number of User Reviews for Laptop Products
def get_laptop_review_count(soup):
    try:
        # Extracting the number of user reviews
        review_count = soup.find("span", attrs={'id': 'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""  # In case of attribute error, return an empty string	

    return review_count

In [43]:
# Function to extract Availability Status for Laptop Products
def get_laptop_availability(soup):
    try:
        # Extracting the availability status
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"  # If availability information is not found, set to "Not Available"	

    return available

In [44]:
if __name__ == '__main__':
    # Adding user agent 
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.5,ar;q=0.5'  # Including both English and Arabic
    }
    # The webpage URL
    URL = "https://www.amazon.com/s?k=laptop&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetching links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

    # Storing the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    d = {"Laptop Model": [], "Price": [], "Rating": [], "Number of Reviews": [], "Availability": []}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to extract all necessary product information
        d['Laptop Model'].append(get_laptop_title(new_soup))
        d['Price'].append(get_laptop_price(new_soup))
        d['Rating'].append(get_laptop_rating(new_soup))
        d['Number of Reviews'].append(get_laptop_review_count(new_soup))
        d['Availability'].append(get_laptop_availability(new_soup))

    # Creating a DataFrame from the extracted data
    laptops_df = pd.DataFrame.from_dict(d)
    
    # Dropping rows with missing titles
    laptops_df['Laptop Model'].replace('', np.nan, inplace=True)
    laptops_df = laptops_df.dropna(subset=['Laptop Model'])
    
    # Saving the DataFrame to a CSV file
    laptops_df.to_csv("laptops_data.csv", header=True, index=False)


C:\Users\20155\AppData\Local\Temp\ipykernel_3532\2669796281.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  laptops_df['Laptop Model'].replace('', np.nan, inplace=True)


In [45]:
laptops_df

,Laptop Model,Price,Rating,Number of Reviews,Availability
0,Auusda Laptop Computer with 16GB DDR4 RAM & 51...,332.49$,3.8 out of 5 stars,188 ratings,In Stock
1,"BiTECOOL 15.6"" Windows 11 Laptop, Intel Core i...",299.00$,4.5 out of 5 stars,168 ratings,In Stock
2,Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6...,259.99$,4.3 out of 5 stars,"1,775 ratings",Only 1 left in stock - order soon.
3,"HP 17 Laptop, 17.3” HD+ Display, 11th Gen Inte...",105.99$,4.3 out of 5 stars,584 ratings,Only 1 left in stock - order soon.
4,"HP Newest 14"" Ultral Light Laptop for Students...",259.99$,4.2 out of 5 stars,"1,828 ratings",Only 1 left in stock - order soon.
5,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",186.60$,3.9 out of 5 stars,"1,308 ratings",In Stock
6,Leudaich 14'’ Laptop Monitor Extender for Dual...,179.80$,4.6 out of 5 stars,143 ratings,In Stock
7,"HP Notebook Laptop, 15.6"" HD Touchscreen, Inte...",455.00$,4.5 out of 5 stars,960 ratings,In Stock
8,"Dell Inspiron 3511 Laptop, 15.6"" Full HD Touch...",529.00$,4.0 out of 5 stars,398 ratings,Only 4 left in stock - order soon.
9,"Lenovo V15 Laptop, 15.6"" FHD Display, AMD Ryze...",22.99$,4.4 out of 5 stars,234 ratings,Not Available
